In [1]:
import torch.nn as nn
import torch.optim as optim
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from ast import literal_eval
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "9"

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
num_workers=1
batch_size=1000
lr=0.01
num_epoch=200

In [3]:
num_level=1
depth_batch=1
hide_neuron=32

annotation_train = 'data/data_se/data_train.txt'
annotation_val = 'data/data_se/data_test.txt'
var_size=1370+1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
class MyDataset(Dataset):
    def __init__(self, annotation):
        self.landmarks_frame = pd.read_csv(annotation)
    def __len__(self):
        return len(self.landmarks_frame)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        i_data = literal_eval(self.landmarks_frame.iloc[idx, 1])

        return i_data
train_dataset = MyDataset(annotation_train)
val_dataset = MyDataset(annotation_val)

In [5]:
def pad_length(b):
    # build vectorized sequence
    v = b
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=var_size-1) for t in v]),
    )

In [6]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, drop_last=True, shuffle=True,collate_fn=pad_length,)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, drop_last=True, shuffle=False,collate_fn=pad_length)

In [7]:
class MyLSTM(nn.Module):
    def __init__(self, output_size, hidden_size, n_layers=1):
        super(MyLSTM, self).__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.LSTM = nn.LSTM(self.output_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.output_size)
    def forward(self,x,hidden):
        out, (h, c)  = self.LSTM(x,hidden)
        out = self.dropout(out)
        x = self.fc(out)
        x = self.fc(out)
        return x, (h, c)

In [8]:
net=MyLSTM(var_size,hide_neuron,num_level)
net=net.to(device)
Loss = nn.CrossEntropyLoss(ignore_index=var_size-1)
optimizer = optim.AdamW(net.parameters(), lr=lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.5,patience=5)

In [9]:
def micro_batch(input,index=0,input_size=1,var_size=1):
    sting=input[:,index:index+input_size]#.type(torch.int32)
    label_ind=input[:,index+1:index+input_size+1]#.type(torch.int32)
    size=np.array([sting.shape[0]*sting.shape[1],var_size])
    batch=np.zeros(size)
    for i in range(sting.shape[0]):
        for j in range(sting.shape[1]):
            a=sting[i][j]
            batch[i+j][a]+=1.0
    label=torch.Tensor(label_ind).reshape(-1)
    batch=torch.Tensor(batch)
    return batch, label
    

In [10]:
num_epoch=80
batch_size=10

In [11]:
best_Loss_regression=np.inf
for iter in range(num_epoch):
    for dataloader, phase in zip([train_dataloader, val_dataloader], ["T", "V"]):
        eposh_loss=0
        if phase == "T": net.train()
        if phase == "V": net.eval()
        for [data] in dataloader:
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == "T"):
                total_loss = torch.tensor([0.0], requires_grad=phase == "T").to(device)
                h0 = torch.zeros(num_level,  hide_neuron,device=device,requires_grad=True)
                c0 = torch.zeros(num_level, hide_neuron,device=device,requires_grad=True)
                for i in range(data.shape[1]-depth_batch):
                    input_batch, target=micro_batch(data,i,depth_batch,var_size)
                    output, (h0, c0) = net(input_batch.to(device), (h0, c0))
                    target = target.long().to(device)
                    loss=Loss(output,target)
                    total_loss=total_loss+loss
                if phase == 'T':
                    # Вычислить градиенты
                    total_loss.backward()
                    # Обновить веса
                    optimizer.step()
                eposh_loss+=total_loss.item()
        if phase == "T": 
            print(f"{iter} {phase} {eposh_loss / len(dataloader):0.3f}", end=" ")
        else:
            scheduler.step(eposh_loss)
            print(f"{phase} {eposh_loss / len(dataloader):0.3f} {optimizer.param_groups[0]['lr']}")
            if eposh_loss < best_Loss_regression:
                best_model_wts = net.state_dict()
                best_model_ind = iter
                best_Loss_regression = eposh_loss
net.load_state_dict(best_model_wts)
torch.save(best_model_wts, 'model_lstm.tar')
print(best_model_ind, best_Loss_regression / len(val_dataloader))

0 T 53.211 V 45.163 0.01
1 T 46.720 V 43.034 0.01
2 T 44.098 V 38.592 0.01
3 T 39.590 V 33.486 0.01
4 T 33.577 V 28.798 0.01
5 T 29.453 V 25.026 0.01
6 T 26.102 V 22.741 0.01
7 T 24.434 V 22.020 0.01
8 T 22.587 V 21.377 0.01
9 T 21.846 V 21.415 0.01
10 T 21.399 V 20.874 0.01
11 T 20.722 V 20.775 0.01
12 T 20.470 V 20.670 0.01
13 T 19.966 V 20.556 0.01
14 T 19.892 V 20.468 0.01
15 T 19.445 V 20.423 0.01
16 T 19.014 V 20.189 0.01
17 T 19.062 V 20.332 0.01
18 T 18.974 V 20.185 0.01
19 T 19.085 V 20.425 0.01
20 T 18.605 V 20.285 0.01
21 T 18.899 V 20.184 0.01
22 T 18.595 V 20.604 0.01
23 T 18.631 V 20.203 0.01
24 T 18.467 V 20.166 0.01
25 T 18.497 V 20.264 0.01
26 T 18.291 V 20.290 0.01
27 T 18.365 V 20.408 0.01
28 T 18.052 V 20.308 0.01
29 T 18.142 V 20.436 0.01
30 T 18.126 V 20.200 0.005
31 T 17.492 V 20.198 0.005
32 T 17.408 V 20.213 0.005
33 T 17.419 V 20.110 0.005
34 T 17.337 V 20.254 0.005
35 T 17.568 V 20.212 0.005
36 T 17.272 V 20.040 0.005
37 T 17.554 V 20.020 0.005
38 T 17.421 V 

In [12]:
n = 0
trye = 0
trye_3 = 0
trye_5 = 0
trye_10 = 0
trye_50 = 0
trye_500 = 0
data_test = pd.read_csv(annotation_val)
net.eval()
for d in data_test['name_id'].values:
    mas = literal_eval(d)
    h0 = torch.zeros(num_level,  hide_neuron,device=device,)
    c0 = torch.zeros(num_level, hide_neuron,device=device,)
    label=mas[-1]
    input_v=mas[:-1]
    
    for i, m in enumerate(input_v):
        buf = np.zeros((var_size),dtype = np.float32)
        buf[m] +=1
        inpyt = torch.tensor([buf]).to(device)
        with torch.set_grad_enabled(False):
            out, (h0, c0) = net(inpyt, (h0, c0))
    trye += label == out.argmax()
    trye_3 += label in out.argsort(descending=True)[0][:3]
    trye_5 += label in out.argsort(descending=True)[0][:5]
    trye_10 += label in out.argsort(descending=True)[0][:10]
    trye_50 += label in out.argsort(descending=True)[0][:50]
    trye_500 += label in out.argsort(descending=True)[0][:500]
    n+=1
print('Acc top 1', trye / n)
print('Acc top 3',trye_3 / n)
print('Acc top 5',trye_5 / n)
print('Acc top 10',trye_10 / n)
print('Acc top 50',trye_50 / n)
print('Acc top 500',trye_500 / n)

/tmp/ipykernel_30793/3158143806.py:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  inpyt = torch.tensor([buf]).to(device)


Acc top 1 tensor(0.1990, device='cuda:0')
Acc top 3 0.4018213772168078
Acc top 5 0.5301965170154976
Acc top 10 0.6914043776961176
Acc top 50 0.9174788304841028
Acc top 500 0.9906534590190126


In [13]:
n = 0
trye = 0
trye_3 = 0
trye_5 = 0
trye_10 = 0
trye_50 = 0
trye_500 = 0
data_test = pd.read_csv(annotation_val)
net.eval()
for d in data_test['name_id'].values:
    mas = literal_eval(d)
    for i, m in enumerate(mas):
        h0 = torch.zeros(num_level,  hide_neuron,device=device,)
        c0 = torch.zeros(num_level,  hide_neuron,device=device,)
        label=m
        input_v=mas[:i]+mas[i+1:]
        for r in input_v:
            buf = np.zeros((var_size),dtype = np.float32)
            buf[r] +=1
            inpyt = torch.tensor([buf]).to(device)
            with torch.set_grad_enabled(False):
                out, (h0, c0) = net(inpyt, (h0, c0))
        trye += label == out.argmax()
        trye_3 += label in out.argsort(descending=True)[0][:3]
        trye_5 += label in out.argsort(descending=True)[0][:5]
        trye_10 += label in out.argsort(descending=True)[0][:10]
        trye_50 += label in out.argsort(descending=True)[0][:50]
        trye_500 += label in out.argsort(descending=True)[0][:500]
        n+=1
print('Acc top 1', trye / n)
print('Acc top 3',trye_3 / n)
print('Acc top 5',trye_5 / n)
print('Acc top 10',trye_10 / n)
print('Acc top 50',trye_50 / n)
print('Acc top 500',trye_500 / n)

Acc top 1 tensor(0.1002, device='cuda:0')
Acc top 3 0.22203007750980452
Acc top 5 0.30942161010406694
Acc top 10 0.46261927554581106
Acc top 50 0.8037550566655344
Acc top 500 0.9542043664885896


In [14]:
save_name = "/workspace/prj/study/buy_with_this_product/res_LSTM/"+ 'LSTM_ACC top1-' + str(round(int(trye / n),4)) + '.tar'
torch.save({
            'state_dict': net.state_dict(),
            }, save_name)